In [1]:
import sys
sys.path.append('../')
import uPVT.PVT as PVT
import Beggs_Brill_class as BB

import plotly.plotly as py
import plotly.graph_objs as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools

init_notebook_mode(connected=True)



fluid = PVT.FluidStanding()



hydr_cor = BB.Beggs_Brill_cor()

p_initial_mpa = 15
t_initial_c = 70
h_initial_m = 1000
step_m = 10
step_cm = 0.03

h_list = []
p_list = []
t_list = []

PT = BB.PT(p_initial_mpa,t_initial_c)



h_list = [h_initial_m]
p_list = [p_initial_mpa]
t_list = [t_initial_c]
for i in range(100):
    if PT.p_mpa > 0:
        
        fluid.calc(PT.p_mpa * 10,PT.T_C)
        hydr_cor.print_all = False
        hydr_cor.mu_gas_pasec = fluid.mu_gas_cP 
        hydr_cor.mu_oil_pasec = fluid.mu_oil_cP 
        hydr_cor.rho_oil_kgm3 = fluid.rho_oil_kgm3
        hydr_cor.gas_rate_on_surface_m3day = hydr_cor.oil_rate_on_surface_m3day * fluid.rsb_m3m3
        hydr_cor.Rs_m3m3 = fluid.rs_m3m3
        hydr_cor.gas_formation_volume_factor_m3m3 = fluid.bg_m3m3
        hydr_cor.oil_formation_volume_factor_m3m3 = fluid.bo_m3m3
        grad_pam = hydr_cor.calc_grad(PT)
        p = p_list[-1] - step_m * grad_pam / 10**6
        t = t_list[-1] - step_m * step_cm
        h = h_list[-1] - step_m
    if p > 0:
        p_list.append(p)
    else:
        p_list.append(p_list[-1])
    t_list.append(t)
    h_list.append(h)
    PT.p_mpa = p
    PT.T_C = t
    hydr_cor.pressure_pa = p * 10**6
    hydr_cor.temperature_c = t
    

0.04113596980323393
0.04113596980323393


In [2]:
tracep = go.Scattergl(
    x = p_list,
    y = h_list,
    name = 'P_MPa',
    mode = 'markers'
)

tracet = go.Scattergl(
    x = t_list,
    y = h_list,
    name = 'T_C',
    mode = 'markers'
)


data = [tracep, tracet]

layout = dict(title = 'БеггсБрилл снизу вверх, H(P,T)',  yaxis=dict(autorange='reversed'))

fig = dict(data=data, layout=layout)

iplot(fig, filename='basic-scatter')